In [1]:
import re
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.utils import shuffle

This is code to get a trivial model running.

No train/validation split here, using all data to train the model.

Also make a prediction and verify the exact meaning of the loss, and mean absolute error.

In [3]:
# Load simulated data:
np.random.seed(11)

# Training set should be one concatenated shuffled list of simulated examples
bounds_files = glob.glob("../data/simulData/*_bmBounds.tab")
intensities_files = [re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
                     for bounds_file in bounds_files]
y, x = shuffle(
    np.concatenate([np.loadtxt(bounds_file) for bounds_file in bounds_files]),
    np.concatenate([np.loadtxt(intensities_file) for intensities_file in intensities_files])
)

In [4]:
def overlap_metric(y_true, y_predicted):
    y_pred_round = tf.round(y_predicted)
    max_start = tf.math.maximum(y_true[:, 0], y_pred_round[:, 0])
    min_end = tf.math.minimum(y_true[:, 1], y_pred_round[:,1])
    overlap = tf.math.maximum(0.0, min_end + 1 - max_start)
    longest_bounds = tf.math.maximum(
        y_true[:, 1] + 1 - y_true[:, 0], 
        y_pred_round[:, 1] + 1 - y_pred_round[:, 0]
        )
    return overlap / longest_bounds

In [ ]:
model = tf.keras.Sequential([
        layers.Dense(40, activation='relu'),
        layers.Dense(2, activation='linear')
    ])
model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    loss='mse', 
    metrics=['mae', overlap_metric]
    )

In [ ]:
history = model.fit(x, y, epochs=1000, batch_size=8192, validation_split=0.33)

Train on 60299 samples, validate on 29701 samples
Epoch 1/1000
60299/60299 [==============================] - 1s 13us/step - loss: 1887.8790 - mean_absolute_error: 34.3398 - overlap_metric: 0.0309 - val_loss: 313.0333 - val_mean_absolute_error: 14.2234 - val_overlap_metric: 0.0773
Epoch 2/1000
60299/60299 [==============================] - 0s 1us/step - loss: 395.3843 - mean_absolute_error: 15.7072 - overlap_metric: 0.0790 - val_loss: 327.5400 - val_mean_absolute_error: 15.0567 - val_overlap_metric: 0.0781
Epoch 3/1000
60299/60299 [==============================] - 0s 1us/step - loss: 235.8862 - mean_absolute_error: 12.2643 - overlap_metric: 0.1016 - val_loss: 204.1591 - val_mean_absolute_error: 11.3907 - val_overlap_metric: 0.0847
Epoch 4/1000
60299/60299 [==============================] - 0s 1us/step - loss: 138.0012 - mean_absolute_error: 9.2717 - overlap_metric: 0.1373 - val_loss: 104.4520 - val_mean_absolute_error: 8.1382 - val_overlap_metric: 0.1347
Epoch 5/1000
60299/60299 [====

Epoch 37/1000
60299/60299 [==============================] - 0s 1us/step - loss: 9.1326 - mean_absolute_error: 2.3042 - overlap_metric: 0.5698 - val_loss: 9.0615 - val_mean_absolute_error: 2.2997 - val_overlap_metric: 0.5768
Epoch 38/1000
60299/60299 [==============================] - 0s 1us/step - loss: 8.9930 - mean_absolute_error: 2.2869 - overlap_metric: 0.5728 - val_loss: 8.9168 - val_mean_absolute_error: 2.2809 - val_overlap_metric: 0.5765
Epoch 39/1000
60299/60299 [==============================] - 0s 1us/step - loss: 8.8630 - mean_absolute_error: 2.2698 - overlap_metric: 0.5754 - val_loss: 8.8008 - val_mean_absolute_error: 2.2652 - val_overlap_metric: 0.5795
Epoch 40/1000
60299/60299 [==============================] - 0s 1us/step - loss: 8.7501 - mean_absolute_error: 2.2553 - overlap_metric: 0.5777 - val_loss: 8.6930 - val_mean_absolute_error: 2.2527 - val_overlap_metric: 0.5799
Epoch 41/1000
60299/60299 [==============================] - 0s 1us/step - loss: 8.6474 - mean_absol

60299/60299 [==============================] - 0s 1us/step - loss: 7.0399 - mean_absolute_error: 2.0319 - overlap_metric: 0.6118 - val_loss: 7.0567 - val_mean_absolute_error: 2.0357 - val_overlap_metric: 0.6193
Epoch 74/1000
60299/60299 [==============================] - 0s 1us/step - loss: 7.0206 - mean_absolute_error: 2.0301 - overlap_metric: 0.6131 - val_loss: 7.0128 - val_mean_absolute_error: 2.0293 - val_overlap_metric: 0.6082
Epoch 75/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.9656 - mean_absolute_error: 2.0215 - overlap_metric: 0.6143 - val_loss: 6.9814 - val_mean_absolute_error: 2.0230 - val_overlap_metric: 0.6144
Epoch 76/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.9565 - mean_absolute_error: 2.0204 - overlap_metric: 0.6140 - val_loss: 6.9700 - val_mean_absolute_error: 2.0245 - val_overlap_metric: 0.6214
Epoch 77/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.9130 - mean_absolute_error: 2.0

60299/60299 [==============================] - 0s 1us/step - loss: 6.1282 - mean_absolute_error: 1.8991 - overlap_metric: 0.6299 - val_loss: 6.0986 - val_mean_absolute_error: 1.8963 - val_overlap_metric: 0.6237
Epoch 110/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.0371 - mean_absolute_error: 1.8840 - overlap_metric: 0.6337 - val_loss: 6.0262 - val_mean_absolute_error: 1.8817 - val_overlap_metric: 0.6320
Epoch 111/1000
60299/60299 [==============================] - 0s 1us/step - loss: 5.9770 - mean_absolute_error: 1.8726 - overlap_metric: 0.6373 - val_loss: 6.0245 - val_mean_absolute_error: 1.8846 - val_overlap_metric: 0.6414
Epoch 112/1000
60299/60299 [==============================] - 0s 1us/step - loss: 5.9671 - mean_absolute_error: 1.8720 - overlap_metric: 0.6376 - val_loss: 6.0021 - val_mean_absolute_error: 1.8779 - val_overlap_metric: 0.6299
Epoch 113/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.0016 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 5.6863 - mean_absolute_error: 1.8367 - overlap_metric: 0.6355 - val_loss: 5.5481 - val_mean_absolute_error: 1.8118 - val_overlap_metric: 0.6537
Epoch 146/1000
60299/60299 [==============================] - 0s 1us/step - loss: 5.8282 - mean_absolute_error: 1.8664 - overlap_metric: 0.6265 - val_loss: 5.5297 - val_mean_absolute_error: 1.8087 - val_overlap_metric: 0.6553
Epoch 147/1000
60299/60299 [==============================] - 0s 1us/step - loss: 5.5256 - mean_absolute_error: 1.8053 - overlap_metric: 0.6465 - val_loss: 5.5155 - val_mean_absolute_error: 1.8057 - val_overlap_metric: 0.6396
Epoch 148/1000
60299/60299 [==============================] - 0s 1us/step - loss: 5.4886 - mean_absolute_error: 1.7998 - overlap_metric: 0.6474 - val_loss: 5.4889 - val_mean_absolute_error: 1.8027 - val_overlap_metric: 0.6540
Epoch 149/1000
60299/60299 [==============================] - 0s 1us/step - loss: 5.4684 - mean_absolute_error:

...

60299/60299 [==============================] - 0s 1us/step - loss: 3.3241 - mean_absolute_error: 1.3873 - overlap_metric: 0.7256 - val_loss: 3.3904 - val_mean_absolute_error: 1.4171 - val_overlap_metric: 0.7203
Epoch 650/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.3213 - mean_absolute_error: 1.3888 - overlap_metric: 0.7264 - val_loss: 3.1309 - val_mean_absolute_error: 1.3443 - val_overlap_metric: 0.7279
Epoch 651/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.4872 - mean_absolute_error: 1.4341 - overlap_metric: 0.7185 - val_loss: 4.0837 - val_mean_absolute_error: 1.5714 - val_overlap_metric: 0.7034
Epoch 652/1000
60299/60299 [==============================] - 0s 1us/step - loss: 4.3603 - mean_absolute_error: 1.6413 - overlap_metric: 0.6891 - val_loss: 3.4845 - val_mean_absolute_error: 1.4294 - val_overlap_metric: 0.7251
Epoch 653/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.4168 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 4.1463 - mean_absolute_error: 1.5959 - overlap_metric: 0.6889 - val_loss: 3.4146 - val_mean_absolute_error: 1.4295 - val_overlap_metric: 0.7105
Epoch 686/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.4587 - mean_absolute_error: 1.4355 - overlap_metric: 0.7120 - val_loss: 3.1441 - val_mean_absolute_error: 1.3562 - val_overlap_metric: 0.7262
Epoch 687/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.1268 - mean_absolute_error: 1.3446 - overlap_metric: 0.7309 - val_loss: 3.0510 - val_mean_absolute_error: 1.3277 - val_overlap_metric: 0.7326
Epoch 688/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0796 - mean_absolute_error: 1.3301 - overlap_metric: 0.7349 - val_loss: 3.1303 - val_mean_absolute_error: 1.3372 - val_overlap_metric: 0.7329
Epoch 689/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0924 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 3.0405 - mean_absolute_error: 1.3235 - overlap_metric: 0.7360 - val_loss: 3.0776 - val_mean_absolute_error: 1.3428 - val_overlap_metric: 0.7313
Epoch 722/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0027 - mean_absolute_error: 1.3127 - overlap_metric: 0.7379 - val_loss: 2.9403 - val_mean_absolute_error: 1.2997 - val_overlap_metric: 0.7396
Epoch 723/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0067 - mean_absolute_error: 1.3135 - overlap_metric: 0.7383 - val_loss: 3.1449 - val_mean_absolute_error: 1.3415 - val_overlap_metric: 0.7345
Epoch 724/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0570 - mean_absolute_error: 1.3259 - overlap_metric: 0.7363 - val_loss: 2.9377 - val_mean_absolute_error: 1.2986 - val_overlap_metric: 0.7433
Epoch 725/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.2089 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 2.9124 - mean_absolute_error: 1.2905 - overlap_metric: 0.7399 - val_loss: 2.8992 - val_mean_absolute_error: 1.2921 - val_overlap_metric: 0.7431
Epoch 758/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.8797 - mean_absolute_error: 1.2821 - overlap_metric: 0.7417 - val_loss: 2.8779 - val_mean_absolute_error: 1.2846 - val_overlap_metric: 0.7452
Epoch 759/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.8647 - mean_absolute_error: 1.2768 - overlap_metric: 0.7430 - val_loss: 2.9384 - val_mean_absolute_error: 1.2921 - val_overlap_metric: 0.7428
Epoch 760/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0229 - mean_absolute_error: 1.3196 - overlap_metric: 0.7376 - val_loss: 2.8912 - val_mean_absolute_error: 1.2816 - val_overlap_metric: 0.7428
Epoch 761/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0318 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 3.0978 - mean_absolute_error: 1.3414 - overlap_metric: 0.7341 - val_loss: 3.1533 - val_mean_absolute_error: 1.3519 - val_overlap_metric: 0.7351
Epoch 794/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.9630 - mean_absolute_error: 1.3073 - overlap_metric: 0.7396 - val_loss: 3.3578 - val_mean_absolute_error: 1.4244 - val_overlap_metric: 0.7217
Epoch 795/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.1076 - mean_absolute_error: 1.3462 - overlap_metric: 0.7337 - val_loss: 3.6639 - val_mean_absolute_error: 1.4735 - val_overlap_metric: 0.7119
Epoch 796/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.9965 - mean_absolute_error: 1.3129 - overlap_metric: 0.7383 - val_loss: 2.8998 - val_mean_absolute_error: 1.2979 - val_overlap_metric: 0.7415
Epoch 797/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.9380 - mean_absolute_error:

In [ ]:
plt.ylim(0, 5)
plt.plot(range(len(history.history['loss'])), history.history['loss'], history.history['val_loss'])

In [ ]:
plt.plot(range(len(history.history['overlap_metric'])),history.history['overlap_metric'])

In [ ]:
print(history.history.keys()) 

## Final accuracy

In [ ]:
print("Mean absolute error", history.history['val_mean_absolute_error'][-1])
print("Overlap metric", history.history['val_overlap_metric'][-1])